In [26]:
# Import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', None)

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier

# Neural Network
# from tensorflow.keras.optimizers import Adam, SGD
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense, Dropout
# from tensorflow.keras.utils import to_categorical
# from tensorflow.keras.regularizers import l2
# from tensorflow.keras.callbacks import EarlyStopping

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import RandomizedSearchCV, train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

import pickle

In [2]:
# Import Data
df = pd.read_csv('data/baseball_no_nulls_after_dan2.csv') #2015-2020 data
df.iloc[:,1:2] = df.iloc[:,1:2].astype(np.float32)
df.iloc[:,4:] = df.iloc[:,4:].astype(np.float32)

In [4]:
# DO NOT RUN AGAIN
batter_dummies.shape[0] == df.shape[0] # confirmed shape, so now I will just concat immediately

True

In [3]:
# Create A threshold
# counts = df['batter_id'].value_counts()
# repl = counts[counts <= 20].index # 287 of them do not matter to me
# df = pd.concat([df,pd.get_dummies(df['batter_id'].replace(repl, 'uncommon'))], axis=1)
df.head(3)

,Pitcher_name,batter_id,pitch_name,strike_attempt,release_speed,release_pos_x,release_pos_z,stand,p_throws,balls,strikes,pfx_x,pfx_z,plate_x,plate_z,inning,inning_topbot,effective_speed,release_spin_rate,pitch_number,bat_score,fld_score,vx0,vy0,vz0,ax,ay,az,outs_when_up,2s_usage,4s_usage,changeup_usage,curveball_usage,cutter_usage,eephus_usage,fastball_usage,forkball_usage,ball_usage,knucklecurve_usage,knuckleball_usage,pitchout_usage,screwball_usage,sinker_usage,slider_usage,split_usage,unknown_usage
0,Wade Davis,527038.0,4-Seam Fastball,strike,96.099998,-1.92,5.95,1.0,1.0,1.0,2.0,-0.27,1.70,-0.93,1.62,12.0,0.0,95.400002,2463.0,6.0,2.0,7.0,3.075,-139.608994,-9.724,-2.028,34.779999,-11.706,2.0,0.018929,0.47782,0.005354,0.000191,0.293881,0.0,0.0,0.0,0.000765,0.19847,0.0,0.000191,0.0,0.0,0.000382,0.0,0.004015
1,Wade Davis,527038.0,Cutter,strike,93.099998,-1.50,6.02,1.0,1.0,1.0,2.0,0.24,0.72,0.78,1.55,12.0,0.0,92.199997,2705.0,5.0,2.0,7.0,5.156,-135.442001,-7.227,5.004,28.834999,-25.323,2.0,0.018929,0.47782,0.005354,0.000191,0.293881,0.0,0.0,0.0,0.000765,0.19847,0.0,0.000191,0.0,0.0,0.000382,0.0,0.004015
2,Wade Davis,527038.0,4-Seam Fastball,strike,97.000000,-1.37,6.08,1.0,1.0,1.0,2.0,-0.50,1.41,1.32,2.47,12.0,0.0,96.400002,2362.0,4.0,2.0,7.0,8.459,-140.843002,-7.313,-5.412,31.551001,-15.189,2.0,0.018929,0.47782,0.005354,0.000191,0.293881,0.0,0.0,0.0,0.000765,0.19847,0.0,0.000191,0.0,0.0,0.000382,0.0,0.004015


In [5]:
ignore = ['Pitcher_name','batter_id','strike_attempt']

In [6]:
y = df['strike_attempt']
X = df.loc[:,'release_speed':]
X.head()

,release_speed,release_pos_x,release_pos_z,stand,p_throws,balls,strikes,pfx_x,pfx_z,plate_x,plate_z,inning,inning_topbot,effective_speed,release_spin_rate,pitch_number,bat_score,fld_score,vx0,vy0,vz0,ax,ay,az,outs_when_up,2s_usage,4s_usage,changeup_usage,curveball_usage,cutter_usage,eephus_usage,fastball_usage,forkball_usage,ball_usage,knucklecurve_usage,knuckleball_usage,pitchout_usage,screwball_usage,sinker_usage,slider_usage,split_usage,unknown_usage
0,96.099998,-1.92,5.95,1.0,1.0,1.0,2.0,-0.27,1.70,-0.93,1.62,12.0,0.0,95.400002,2463.0,6.0,2.0,7.0,3.075,-139.608994,-9.724,-2.028,34.779999,-11.706,2.0,0.018929,0.47782,0.005354,0.000191,0.293881,0.0,0.0,0.0,0.000765,0.19847,0.0,0.000191,0.0,0.0,0.000382,0.0,0.004015
1,93.099998,-1.50,6.02,1.0,1.0,1.0,2.0,0.24,0.72,0.78,1.55,12.0,0.0,92.199997,2705.0,5.0,2.0,7.0,5.156,-135.442001,-7.227,5.004,28.834999,-25.323,2.0,0.018929,0.47782,0.005354,0.000191,0.293881,0.0,0.0,0.0,0.000765,0.19847,0.0,0.000191,0.0,0.0,0.000382,0.0,0.004015
2,97.000000,-1.37,6.08,1.0,1.0,1.0,2.0,-0.50,1.41,1.32,2.47,12.0,0.0,96.400002,2362.0,4.0,2.0,7.0,8.459,-140.843002,-7.313,-5.412,31.551001,-15.189,2.0,0.018929,0.47782,0.005354,0.000191,0.293881,0.0,0.0,0.0,0.000765,0.19847,0.0,0.000191,0.0,0.0,0.000382,0.0,0.004015
3,93.599998,-1.25,6.06,1.0,1.0,0.0,2.0,0.32,0.80,2.92,2.23,12.0,0.0,92.800003,2724.0,3.0,2.0,7.0,10.094,-135.733994,-5.821,6.079,26.355000,-24.250,2.0,0.018929,0.47782,0.005354,0.000191,0.293881,0.0,0.0,0.0,0.000765,0.19847,0.0,0.000191,0.0,0.0,0.000382,0.0,0.004015
4,97.099998,-1.49,6.08,1.0,1.0,0.0,1.0,-0.76,1.63,0.26,2.74,12.0,0.0,96.000000,2401.0,2.0,2.0,7.0,6.519,-140.992996,-7.138,-9.148,32.250999,-11.956,2.0,0.018929,0.47782,0.005354,0.000191,0.293881,0.0,0.0,0.0,0.000765,0.19847,0.0,0.000191,0.0,0.0,0.000382,0.0,0.004015


In [ ]:
X2 = df.loc[col for col in df.columns if col not in ignore]
X2.head()

In [8]:
X2 = pd.get_dummies(X2, columns=['pitch_name'])

In [8]:
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y, random_state=66, stratify=y)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=66, stratify=y)

In [9]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((2824295, 42), (941432, 42), (2824295,), (941432,))

# Random Forest

---
The features not included are Pitcher name, batter id, and pitch name.

In [25]:
params = ({
    'n_estimators':[100,125,150,175],
    'max_depth':[2,3,4,5],
    'max_features':['log2','auto']
})
    

In [16]:
rf = RandomizedSearchCV(RandomForestClassifier(),
                       param_distributions=params,
                       n_jobs=8,
                       verbose=2,
                       random_state=66)

In [17]:
rf.fit(X_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed: 22.3min
[Parallel(n_jobs=8)]: Done  50 out of  50 | elapsed: 45.7min finished


RandomizedSearchCV(estimator=RandomForestClassifier(), n_jobs=8,
                   param_distributions={'max_depth': [2, 3, 4, 5],
                                        'max_features': ['log2', 'auto'],
                                        'n_estimators': [100, 125, 150]},
                   random_state=66, verbose=2)

In [18]:
rf.score(X_train, y_train)

0.653994713725018

In [19]:
rf.score(X_test, y_test)

0.6541130957944918

In [21]:
rf.best_params_

{'n_estimators': 150, 'max_features': 'auto', 'max_depth': 5}

In [23]:
preds = rf.predict(X_test)

In [24]:
accuracy_score(y_test, preds)

0.6541130957944918

In [30]:
# Using the same parameters but on a GridSearch
rf2 = GridSearchCV(RandomForestClassifier(random_state=66),
                       param_grid=params,
                       n_jobs=8,
                       verbose=2)

In [ ]:
rf2.fit(X_train, y_train)

Fitting 5 folds for each of 32 candidates, totalling 160 fits


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed: 19.4min


These models will included the dummified pitch name at the very least.

In [ ]:
m